# Introduction | Movies plot in russian dataset 

n the notebook, I am attempting to create a simple BiLSTM model capable of generating text in Russian. The model has been trained on a dataset containing movie plots.

## Importing libraries, funcs & data


In [7]:
!pip install pyspark > /dev/null 2>&1
!python -m spacy download ru_core_news_sm > /dev/null 2>&1

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

In [84]:
import os 
import pandas as pd
import tensorflow.keras as keras 
import numpy as np
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import spacy

## Importing dataset


In [51]:
path: str = r'/kaggle/input/movie-plots-from-wikipedia-in-russian/films_data.csv'
df = pd.read_csv(path, sep=',')

## Data Exploration

In [52]:
df.head(10)


,title,type,genre,imdb_rating,summary,plot
0,? (фильм),film,драматический фильм,7.0,«?» (индон. Tanda Tanya) — индонезийский худож...,Основная тема фильма — межрелигиозные отношени...
1,...а пятый всадник – Страх,film,драма военный артхаус,7.2,«…а пятый всадник — Страх» (чеш. …a páty jezde...,Прага во время немецкой оккупации Чехословакии...
2,…и передайте привет ласточкам,film,драма военный,6.8,«…и передайте привет ласточкам» — (чеш. ...a p...,"Конец 1942 года, нацистская тюрьма в Бреслау. ..."
3,«Чудотворец» из Бирюлёва,film,игровое кино,NaN,«Чудотворец из Бирюлёва» — советский короткоме...,Жена ответственного работника Зоя Фёдоровна на...
4,(Не)идеальные роботы,film,комедия фантастика,5.4,«(Не)идеальные роботы» (англ. Robots) — художе...,Действие фильма разворачивается в далёком буду...
5,0-41*,film,документальный драма комедия,8.9,«0-41*» — индийский документальный драматическ...,Действие фильма происходит в небольшом городе ...
6,1 (документальный фильм),film,документальный,7.9,«1» (также известный «1: Жизнь на пределе»; ан...,Фильм начинается с Гран-при Австралии 1996 год...
7,1 % (фильм),film,криминальный,5.3,«1 %» — австралийский криминальный фильм режис...,Название фильма происходит от так называемых «...
8,1 миля до тебя,film,драма мелодрама,6.3,«1 миля до тебя» — американский мелодраматичес...,"Кевин — обычный старшеклассник, отправившийся ..."
9,1 Night in China,film,порнофильм,2.6,"1 Night in China — порнографический фильм, в г...",В фильме присутствуют сцены с путешествием Лор...


In [53]:
df_genre_counts = df.groupby('genre').size().reset_index(name='count_genre')

df_genre_counts_sorted = df_genre_counts.sort_values(by='count_genre', ascending=False)

df_genre_counts_sorted


,genre,count_genre
3343,драма,5234
5107,комедия,2788
6171,мелодрама,886
4257,драматический фильм,661
9967,фильм ужасов,594
...,...,...
10987,этти гарем комедия,1
10990,этти романтическая комедия,1
10992,этти спокон,1
10977,эротический фильм и драматический фильм,1


In [54]:
df.loc[0,'plot']

'Основная тема фильма — межрелигиозные отношения в Индонезии: стране, где конфликты на религиозной почве являются обычным делом, и долгое время продолжается дискриминация и насилие в отношении индонезийцев китайского происхождения.\nВ фильме показана судьба трёх семей, проживающих в деревне неподалёку от Семаранга (Центральная Ява): индонезиец китайского происхождения, буддист Тан Кат Сун (исполнитель роли — Хенки Сулаеман) и его сын Хендра (Рио Деванто); мусульманин Солех (Реза Рахадиан) и его жена, мусульманка Менук (Ревалина Темат); перешедшая из ислама в католицизм Рика (Эндхита) и её сын — мусульманин Аби.\nСун и Хендра владеют китайским рестораном, где подают, в том числе, блюда из свинины, запретной для мусульман; но при этом у ресторана есть много мусульманских клиентов и сотрудников. Чтобы поддерживать хорошие отношения со своими мусульманскими сотрудниками и клиентами, Сун использует для приготовления свинины отдельную посуду, которую он не разрешает использовать для других б

## Data Preprocessing


In [55]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws, udf, size  # Добавлен импорт size
from pyspark.sql.types import ArrayType, StringType
import spacy

spark = SparkSession.builder.appName("Text Processing").getOrCreate()

nlp = spacy.load('ru_core_news_sm', disable=['parser', 'tagger', 'ner'])

def get_tokens(doc_text):
    if doc_text is None or doc_text.strip() == "":
        return []
    tokens = [token.text.lower() for token in nlp(doc_text) if not token.is_space and not token.is_punct]
    return tokens

tokenize_udf = udf(get_tokens, ArrayType(StringType()))

df = spark.read.csv(path, header=True, inferSchema=True)

df_filtered = df.filter((col("genre") == "драма") | (col("genre") == "мелодрама"))

empty_plot_count = df_filtered.filter(col("plot").isNull() | (col("plot") == "")).count()
print(f"Количество строк с пустыми значениями в 'plot': {empty_plot_count}")

df_with_tokens = df_filtered.withColumn("tokens", tokenize_udf(df['plot']))

df_with_tokens_filtered = df_with_tokens.filter(col("tokens").isNotNull() & (size(col("tokens")) > 0))
print(f"Количество строк после токенизации: {df_with_tokens_filtered.count()}")

df_with_tokens_string = df_with_tokens_filtered.withColumn("tokens_string", concat_ws(" ", df_with_tokens_filtered["tokens"]))
df_final = df_with_tokens_string.select("genre", "tokens_string")

df_final.write.csv('/kaggle/working/processed_text_drama_4.csv', header=True, mode='overwrite')
df_final.show(5)

spark.stop()


Количество строк с пустыми значениями в 'plot': 1574


Количество строк после токенизации: 4546


+---------+--------------------+
|    genre|       tokens_string|
+---------+--------------------+
|    драма|трое случайных зн...|
|    драма|действие происход...|
|    драма|тони мусулин 10 л...|
|мелодрама|фильм расскажет о...|
|    драма|действие фильма п...|
+---------+--------------------+
only showing top 5 rows



In [56]:
directory_path = f'/kaggle/working/processed_text_drama_4.csv'

csv_files = [entry.name for entry in os.scandir(directory_path) if entry.name.endswith('.csv')][0]
csv_files

'part-00001-c3050277-0bdc-4ea0-8c49-185b482772e6-c000.csv'

In [57]:
df = pd.read_csv(f'{directory_path}/{csv_files}',sep=',')
df.head(6)

,genre,tokens_string
0,драма,действие происходит в начале 1980-х годов в сс...
1,драма,главный герой фильма учёный биолог доктор итэн...
2,драма,молодые родители алла и роберт собираются уеха...
3,драма,молодой человек по имени якоб фон гунтен посту...
4,драма,матёрый циничный но не слишком удачливый полит...
5,мелодрама,история любви скрипача виртуоза холгера брандт...


In [58]:
df.shape

(1420, 2)

In [59]:
df.loc[3, 'tokens_string']

'молодой человек по имени якоб фон гунтен поступает в институт беньямента где он собирается пройти курс обучения на домашнего слугу институтом руководит иоханнес беньямента занятия проводит его сестра лиза единственная учительница в классе ещё несколько человек мужчин молодого или среднего возраста они живут в одной комнате однако якоба поселяют отдельно в маленькой комнате на чердаке занятия состоят в основном из повторения и заучивания наизусть правил поведения слуги'

In [60]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [61]:
df.count()

genre            1420
tokens_string    1420
dtype: int64

In [66]:
df = df[:int(0.5*len(df))]
df

,genre,tokens_string
0,драма,действие происходит в начале 1980-х годов в сс...
1,драма,главный герой фильма учёный биолог доктор итэн...
2,драма,молодые родители алла и роберт собираются уеха...
3,драма,молодой человек по имени якоб фон гунтен посту...
4,драма,матёрый циничный но не слишком удачливый полит...
...,...,...
350,драма,конец xix века в канун праздника новруз в расп...
351,драма,токико амамия красивая молодая женщина вынужде...
352,мелодрама,в фильме находит отражение нелёгкий путь студе...
353,драма,история жизни молодой девушки вырвавшейся из г...


In [67]:
min_length = 30
filtered_df = df[df['tokens_string'].str.split().str.len() > min_length]
filtered_df.reset_index(inplace=True, drop=True)
filtered_df.shape

(281, 2)

In [70]:
text_lengths = filtered_df['tokens_string'].str.split().str.len()
average_length = text_lengths.mean()
print(f'Средняя длина текста: {average_length}')


Средняя длина текста: 78.53024911032028


In [68]:
filtered_df.iloc[20,1]

'богатый помещик сурадж пратап сингх джеки шрофф встречает в храме девушку из бедной семьи пуджу и влюбляется в неё он женится на ней и соглашается взять в дом её маленького брата раджу с которым пуджа не хочет расставаться повзрослев раджу салман хан влюбляется в младшую сестру сураджа джоти рамбха с которой он вместе вырос но не все складывается удачно в их семье пуджа никак не может родить ребёнка однажды сурадж встречает прекрасную танцовщицу вайшали швета менон которую делает своей любовницей несмотря на предостережения он во всем потакает своей новой возлюбленной и не обращает внимания на выходки её брата'

In [105]:
texts = filtered_df['tokens_string'].values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)


sequences = tokenizer.texts_to_sequences(texts)

input_sequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequences.append(n_gram_sequence)

In [106]:
sequence_length = 78

In [107]:
input_sequences = pad_sequences(input_sequences, maxlen=sequence_length, padding='pre')


In [108]:
sequences = np.array(input_sequences)

In [109]:
unique_tokens_count = len(tokenizer.word_counts)
print(f"Количество уникальных токенов: {unique_tokens_count}")

Количество уникальных токенов: 9043


In [110]:
X = sequences[:, :-1]
x_len = X.shape[1]
X.shape

(17784, 77)

In [111]:
y = sequences[:, -1]
y.shape

(17784,)

In [112]:
from keras.utils import to_categorical

y = to_categorical(y, num_classes=(unique_tokens_count+1))


In [113]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout, BatchNormalization, Bidirectional

def create_model(vocabulary_size, seq_len):
    model = Sequential()
    
    model.add(Embedding(input_dim=vocabulary_size, output_dim=512))
    
    model.add(Bidirectional(LSTM(units=256, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(units=vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
     
    return model


In [114]:
model = create_model(vocabulary_size=(len(tokenizer.word_index) + 1), seq_len=x_len)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [123]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(buffer_size=1024)

val_size = int(0.3 * len(X)) 
train_size = len(X) - val_size

train_dataset = dataset.take(train_size).batch(16)
val_dataset = dataset.skip(train_size).batch(16)


model.fit(train_dataset, epochs=100)



Epoch 1/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.0441 - loss: 7.2063
Epoch 2/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.0450 - loss: 7.1491
Epoch 3/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.0464 - loss: 7.0842
Epoch 4/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.0469 - loss: 7.0685
Epoch 5/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.0462 - loss: 7.0183
Epoch 6/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.0485 - loss: 7.0043
Epoch 7/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.0469 - loss: 6.9545
Epoch 8/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.0487 - loss: 6.9343
Epoch 9/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.0496 - loss: 6.8755
Epoch 10/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.0455 - loss: 6.8457
Epoch 11/100
779/779 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.0481 - loss: 6.8652
Epoch 12/100
779/77

In [124]:
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation Loss: {loss}\nValidation Accuracy: {accuracy}')

334/334 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.0734 - loss: 9.4837
Validation Loss: 10.661494255065918
Validation Accuracy: 0.06991565227508545


In [125]:
loss, accuracy =  model.evaluate(x=X, y=y)
print(f'Loss: {loss}\nAccuracy: {accuracy}')


556/556 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.1077 - loss: 6.5509
Loss: 7.604384422302246
Accuracy: 0.09570400416851044


In [146]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_text2(model, tokenizer, seq_len, seed_text, num_gen_words, temperature=1.0):

    output_text = []
    input_text = seed_text

    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

        pred_distribution = model.predict(pad_encoded, verbose=0)[0]

        pred_distribution = np.clip(pred_distribution, 1e-9, 1)

        new_pred_distribution = np.power(pred_distribution, (1 / temperature)) 
        new_pred_distribution = new_pred_distribution / new_pred_distribution.sum()

        choices = range(new_pred_distribution.size)
        pred_word_ind = np.random.choice(a=choices, p=new_pred_distribution)

        pred_word = tokenizer.index_word.get(pred_word_ind, '<UNK>')

        input_text += ' ' + pred_word
        output_text.append(pred_word)

    return ' '.join(output_text)




Сгенерированный текст: он в в молодого жена она в сына рода осло в одну своим несмотря на ни со и наконец несмотря на юная девушки агент дочери придётся избрала они несколько за насмешках обнаруживает с тем своим и девушка пытается благородство и


In [224]:
text_sample = filtered_df.loc[1,'tokens_string']
seed_text= text_sample[:49]
seed_text

'молодые родители алла и роберт собираются уехать '

In [226]:
generated_text = generate_text2(model=model, 
                                tokenizer=tokenizer,
                                seq_len=len_0, 
                                seed_text=seed_text, 
                                num_gen_words=10, 
                                temperature=1.0)

print(f"Сгенерированный текст: {seed_text} {generated_text}")

Сгенерированный текст: молодые родители алла и роберт собираются уехать  на мастером и у родители барка отправляет от внука году


In [222]:
text_sample = filtered_df.loc[2, 'tokens_string']
seed_text= text_sample[:15]
seed_text

'молодой человек'

In [174]:
generated_text = generate_text2(model=model, 
                                tokenizer=tokenizer,
                                seq_len=len_0, 
                                seed_text=seed_text, 
                                num_gen_words=20, 
                                temperature=1.0)

print(f"Сгенерированный текст: {seed_text} {generated_text}")

Сгенерированный текст: молодой человек и же происходит и годы охватывает почему и перед бегством и вскоре влюбляется в старше матери актрисы время которые то


In [236]:
text_sample = filtered_df.loc[6, 'tokens_string']
seed_text= text_sample[:19]
seed_text

'в фильме повествует'

In [266]:
generated_text = generate_text2(model=model, 
                                tokenizer=tokenizer,
                                seq_len=len_0, 
                                seed_text=seed_text, 
                                num_gen_words=20, 
                                temperature=1.0)

print(f"Сгенерированный текст: {seed_text} {generated_text}")

Сгенерированный текст: в фильме повествует о встречает же живёт войны приезжает его всё семьи которая пытаясь родственников при центра своей гранатомёта годов в 1917 а


In [277]:
text_sample = filtered_df.loc[15, 'tokens_string']
seed_text= text_sample[:6]
seed_text

'парень'

In [287]:
generated_text = generate_text2(model=model, 
                                tokenizer=tokenizer,
                                seq_len=len_0, 
                                seed_text=seed_text, 
                                num_gen_words=20, 
                                temperature=1.0)

print(f"Сгенерированный текст: {seed_text} {generated_text}")

Сгенерированный текст: парень встречает село офицер вокруг смертью мать сам джонни грозит оказался и жизни жена и работа них к смерти когда возможное
